# Stable Diffusion - Concepts

* https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features
* https://techhenzy.com/stable-diffusion-ultimate-beginners-guide/
* https://www.reddit.com/r/StableDiffusion/comments/z6y6n4/comment/iy4adq5/?utm_source=share&utm_medium=web2x&context=3

## Prompts

### Negative Prompt

### Attention

* a `(word)` - increase attention to word by a factor of 1.1
* a `((word))` - increase attention to word by a factor of 1.21 (= 1.1 * 1.1)
* a `[word]` - decrease attention to word by a factor of 1.1
* a `(word:1.5)` - increase attention to word by a factor of 1.5
* a `(word:0.25)` - decrease attention to word by a factor of 4 (= 1 / 0.25)
* a `\(word\)` - use literal () characters in prompt

### Refiner

This secondary model is designed to process the 1024×1024 SD-XL image near completion, to further enhance and refine details in your final output picture. As of version 1.6.0, this is now implemented in the webui natively.

### VAE

A VAE is a variational autoencoder.

An autoencoder is a model (or part of a model) that is trained to produce its input as output. By giving the model less information to represent the data than the input contains, it's forced to learn about the input distribution and compress the information. A stereotypical autoencoder has an hourglass shape - let's say it starts with 100 inputs and reduces it to 50 then 20 then 10 (encoder) and then 10 to 20 to 50 to 100 (decoder). The 10 dimensions that the encoder produces and the decoder consumes are called the latent representation.

Autoencoders can be a powerful paradigm and can be trained in an unsupervised way (without needing to label data since we only need the input data). However, if we want to sample from the input distribution, a vanilla autoencoder makes this difficult or impossible. One variation on the autoencoder is the variational autoencoder where the latent is normally distributed, which allows for the output distribution to be sampled from.

SD is somewhat unique in the vision class of diffusion models in that the diffusion process operates in the autoencoder space instead of pixel space. This makes the diffusion process more computationally efficient / memory efficient compared to a vanilla pixel space diffusion model. One other related technique some models use is to start the diffusion at a lower spatial resolution and progressively upscale to save compute.

In practice, in SD, the VAE is pretty aggressive and the dataset is filtered (indirectly through the aesthetic score) which removes images with a lot of text. This combined with the autoencoder is a significant reason SD struggles more with producing text than models like Dall-e.

From the above, an autoencoder is essential in SD. Generally speaking, there's no reason to modify the autoencoder unless the image distribution you're training on is dramatically different than the natural images given to SD. In this case, you'd likely need to retrain all parts of the model (or at least the unet). One example case where this might be useful is if you wanted to train an audio diffuser using the same components as SD but on "pixel" data from a spectrogram.

### Sampling Steps


The AI model starts from random noise and then iteratively denoises the image until you get your final image. This modifier decides how many denoising steps it will go through. Default is 50, which is perfect for most scenarios. For reference, at around 10 steps you have generally a good idea of the composition and whether you will like that image or not, at around 20 it becomes very close to finished. If cfg_scale and sampler are at default settings, then the difference 20 steps and 150 (the maximum) is often times hard to tell. So if you want to increase the speed at which your images are generated try lowering the steps. Increasing steps also often adds finer detail and fixes artifacts (often but not always). Example prompt is: !dream “Your prompt here” -s 20

### Upscaler

### Hires. fix


### CFG Scale

Simply put, the CFG scale (classifier-free guidance scale) or guidance scale is a parameter that controls how much the image generation process follows the text prompt. The higher the value, the more the image sticks to a given text input.

### Seed

This is the key to creating these pseudo-variations. If you reuse a prompt with the same seed (as well as all other settings such as steps, cfg scale, etc), you will get exactly the same image

### Sampling Method

https://www.reddit.com/r/StableDiffusion/comments/wwm2at/sampler_vs_steps_comparison_low_to_mid_step_counts/